# Exploration

Modelling of neutrinos via decays in astrophysical sources requires some generally applicable steps. First, the relevant portions of the source environment have to be understood. This foundation is translated to energetic hadron spectra for injection and production. Finally, we obtain the neutrino spectrum from decaying the previous particles, allowing ultrahigh energy cosmic rays to propagate over large distances.

##### Importing Packages

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sci
import scipy.constants as const

##### Setting Display Options

In [2]:
%config InlineBackend.figure_formats = ['svg']

mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color = ['olivedrab', 'steelblue', 'firebrick', 'goldenrod'])
mpl.rcParams['figure.figsize'] = [8, 5]
mpl.rcParams['figure.constrained_layout.use'] = True
mpl.rcParams['legend.frameon'] = False
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True

## Table of Contents
* [1 Environments of Astrophysical Sources](#1-Environments-of-Astrophysical-Sources)
  * [1.1 Magnetars](#1.1-Magnetars)
  * [1.2 Active Galactic Nuclei](#1.2-Active-Galactic-Nuclei)
* [2 Hadronic Spectra in the Region of Origin](#2-Hadronic-Spectra-in-the-Region-of-Origin)
  * [2.1 Injection](#2.1-Injection)
    * [2.1.1 Magnetar](#2.1.1-Magnetar)
    * [2.1.2 Active Galactic Nucleus](#2.1.2-Active-Galactic-Nucleus)
  * [2.2 Production](#2.2-Production)
    * [2.2.1 Magnetar](#2.2.1-Magnetar)
    * [2.2.2 Active Galactic Nucleus](#2.2.2-Active-Galactic-Nucleus)
* [3 Neutrino Spectrum at the Detector](#3-Neutrino-Spectrum-at-the-Detector)
  * [3.1 Decay Processes](#3.1-Decay-Processes)
  * [3.2 Measurement](#3.2-Measurement)



## 1 Environments of Astrophysical Sources


### 1.1 Magnetars

##### Radiative Energy Losses

* $E \propto \omega^2$
* $\dot{E} \propto \omega\,\dot{\omega}$
* $\dot{\omega} \propto \omega^n$

##### Braking Index

* Pure Gravitational or Electromagnetic Quadrupoles
  * $n = 5$
* Pure Magnetic Dipole
  * $n = 3$

##### Characteristic Time Scale

* $\tau = -\tfrac{E_0}{\dot{E}_0}$

##### Conversion of Rotational Energy into Dipole Radiation

* $E = \tfrac{1}{2} I \Omega^2$
* $\dot{E} = I \Omega\dot{\Omega} = -K\Omega^4$

##### Spindown Time
* $t_{SD} = \dfrac{I_{NS}}{2K\Omega_0^2}$

##### Time Dependence
* $\Omega = \Omega_0 \left( 1 + \tfrac{t}{t_{SD}} \right)^{-1/2}$

##### Luminosities of Tilted Dipole Rotator

* Vacuum
  * $L = \dfrac{2\mu_{NS}^2\Omega_0^4\sin^2\!\chi}{3c^3} \left( 1 + \tfrac{t}{t_{SD}} \right)^{-2}$
* Force Free
  * $L = \dfrac{\mu_{NS}^2\Omega_0^4 \left( 1 + \sin^2\!\chi \right)}{c^3} \left( 1 + \tfrac{t}{t_{SD}} \right)^{-2}$

##### Dipole Moment of a Homogeneously Magnetized Sharply Bound Sphere

* $\mu_{NS}^2 = \dfrac{B_{NS}^2R_{NS}^6}{4}$

##### Light Cylinder

* $R_{LC} = \tfrac{c}{\Omega}$

##### Charged Particle Acceleration

* Magnetic Flux
  * $\Psi = \mu_{NS}R_{LC}^{-1}$
* Potential Drop
  * $\Phi = \Psi R_{LC}^{-1}$
* Maximum Kinetic Energy
  * $E_\text{max} = e \Phi$
* Monochromatic Inefficiency Dampened Energy
  * $E_M = f_\text{acc} E_\text{max}$

##### Discrete Proton Spectrum

* Idealized Space Charge Density
  * $\rho = \dfrac{B_{NS}\,\Omega}{2\pi c}$
* Proton Number Density
  * $n_p = \tfrac{\rho}{e}$
* Polar Cap
  * $R_{PC} = R_{NS} \sqrt{\dfrac{R_{NS}}{R_{LC}}}$
* Integrating
  * $dN_p = n_p \,dx\,dy\,dz$
  * $d\dot{N}_p = n_p \,dx\,dy\tfrac{dz}{dt} = c n_p \,dx\,dy$
  * $\dot{N}_p = 2\pi c n_p R_{PC}^2 = 2\pi n_p R_{NS}^3 \Omega$
* Spectrum
  * $\dfrac{d\dot{N}_p}{dE_p} = \dfrac{B_{NS}R_{NS}^3 \Omega^2}{ec \,\bigl(\, 1 + \tfrac{t}{t_{SD}} \,\bigl)} \,\delta(E_p - E_M)$

### 1.2 Active Galactic Nuclei

### 1.2 Active Galactic Nuclei

## 2 Hadronic Spectra in the Region of Origin

### 2.1 Injection

#### 2.1.1 Magnetar

#### 2.1.2 Active Galactic Nucleus

### 2.2 Production

#### 2.2.1 Magnetar

#### 2.2.2 Active Galactic Nucleus

## 3 Neutrino Spectrum at the Detector

### 3.1 Decay Processes

### 3.2 Measurement